In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from nltk.corpus import stopwords 
import nltk
import re
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances

C:\Users\uroot\anaconda3\envs\ml\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#nltk.download('stopwords')

In [3]:
stop = set(['?',',',':','@']) 

In [4]:
ROOT = '../data/'

QA_dict = {}

In [5]:
os.listdir(ROOT)

['CDC.xlsx',
 'Coronavirus (COVID-19) frequently asked questions _ CDC.pdf',
 'covidquestionsca.xlsx',
 'EAC.xlsx',
 'JHU.xlsx',
 'MOHKE.xlsx',
 'sources.txt',
 'WHO.xlsx']

In [6]:
paths = [ROOT+d for d in os.listdir(ROOT) if d.endswith(".xlsx")]

In [7]:
paths

['../data/CDC.xlsx',
 '../data/covidquestionsca.xlsx',
 '../data/EAC.xlsx',
 '../data/JHU.xlsx',
 '../data/MOHKE.xlsx',
 '../data/WHO.xlsx']

In [8]:
data = [ pd.read_excel(p, names = ['q','a']) for p in paths]

In [9]:
QA = pd.DataFrame()

In [10]:
for d in data[:]:
    if d.shape[0]>0:
        print(d.shape)
        QA = pd.concat((QA,d),axis = 0)

(116, 2)
(10, 2)
(26, 2)
(86, 2)
(85, 2)


In [11]:
QA.shape

(323, 2)

In [12]:
QA.sample(10)

,q,a
7,How does COVID-19 spread?,People can catch COVID-19 from others who have...
81,What should we do to protect employees at busi...,The CDC has some specific guidance on this. It...
7,How can I prevent myself from getting infected?,The best way to prevent an infection is to avo...
82,Does the science support limiting the number o...,Yes. We should be practicing social distancing...
11,Can someone who is released from COVID-19 quar...,Someone who has been released from COVID-19 qu...
29,Should contact lens wearers take special preca...,Currently there is no evidence to suggest cont...
84,Does WHO recommend that all international mass...,No. As each international mass gathering is di...
14,What can I do to protect myself and prevent th...,Maintain at least 1 metre (3 feet) distance be...
0,What is a coronavirus?,Coronaviruses are a family of viruses that typ...
9,Is there vaccine availabele?,"At this time, there is no vaccine for COVID-19..."


In [13]:
 snow = nltk.stem.SnowballStemmer('english')

In [14]:
def clean(q_a):
    temp = []
    snow = nltk.stem.SnowballStemmer('english')
    for sentence in q_a:
        sentence = str(sentence)
        sentence = sentence.lower()                 # Converting to lowercase
        cleanr = re.compile('<.*?>')
        sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
        sentence = re.sub(r'[?|!|\'|"|#]\d',r'',sentence)
        sentence = re.sub(r'[.|,|)|(|\|/]@',r' ',sentence)        #Removing Punctuations
        words = [snow.stem(word) for word in sentence.split() if word not in stop]   # Stemming and removing stopwords
        temp.append(words)
    q_a = temp  
    return q_a

In [15]:
def clean_col(c):
    sent = []
    for row in c:
        sequ = ''
        for word in row:
            sequ = sequ + ' ' + word
        sent.append(sequ)
    c = sent
    return c

In [16]:
q_new =  clean_col(clean(QA['q']))
a_new =  clean_col(clean(QA['a']))

In [17]:
QA['q_new'] =  q_new
QA['a_new'] = a_new

In [18]:
QA.head()

,q,a,q_new,a_new
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...,what is a novel coronavirus?,a novel coronavirus is a new coronavirus that...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza...",whi is the diseas be call coronavirus diseas ...,"on februari 11, 2020 the world health organ a..."
2,Why might someone blame or avoid individuals a...,People in the U.S. may be worried or anxious a...,whi might someon blame or avoid individu and ...,peopl in the u.s. may be worri or anxious abo...
3,How can people help stop stigma related to COV...,People can ght stigma by providing social supp...,how can peopl help stop stigma relat to covid...,peopl can ght stigma by provid social support...
4,Why do some state’s COVID-19 case numbers some...,CDC’s overall case numbers are validated throu...,whi do some state covid-19 case number someti...,cdc overal case number are valid through a co...


In [19]:
w2v_data = QA['q_new']

In [20]:
splitted = []
for row in w2v_data: 
    splitted.append([word for word in row.split()]) 

In [21]:
train_w2v = Word2Vec(splitted,min_count=1,size=100, workers=4)

In [22]:
avg_data = []
for row in splitted:
    vec = np.zeros(100)
    count = 0
    for word in row:
        try:
            vec += train_w2v[word]
            count += 1
        except:
            pass
    avg_data.append(vec/count)

C:\Users\uroot\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [23]:
avg_data = np.array(avg_data)

In [24]:
for i,(q , a) in enumerate(zip(QA['q'],QA['a'])):
    QA_dict[i] = [q,a]                   

In [25]:
def wv_test(q):
    tq = [q]
    tq = clean(tq)
    tq = clean_col(tq)
    splitted_test = []
    for row in tq: 
        splitted_test.append([word for word in row.split()]) 
    avg_data_test = []
    for row in splitted_test:
        vec = np.zeros(100)
        count = 0
        for word in row:
            try:
                vec += train_w2v[word]
                count += 1
            except:
                pass
        avg_data_test.append(vec/count)
    return np.array(avg_data_test)

In [26]:
QA['q'].sample(20)

4     What symptoms do coronaviruses typically cause...
23                       How can I help protect myself?
77    How are COVID-19 and influenza viruses different?
68    Is it possible to have the flu and COVID-19 at...
74    Is there evidence for using hydroxychloroquine...
7     Is the loss of smell an early coronavirus indi...
60    How did the first human SARS-CoV-2 infections ...
60    What cleaning products should I use to protect...
19    What can I do to protect myself and prevent th...
21    Is there evidence of a heightened risk to preg...
12                 Why are we seeing a rise in cases?\n
89                  What is CDC doing about COVID-19?\n
32    Are the symptoms of COVID-19 di erent in child...
6     Is there any concern mosquitoes carying the vi...
91    Is it safe to vacuum in a school, business, or...
24            Are COVID toes related to blood clotting?
7                             How does COVID-19 spread?
82    Does the science support limiting the numb

In [ ]:
while True:
    q = input("Q: ")
    q = wv_test(q)
    ranks = np.array([cosine_similarity(q.reshape(1,100),avg_data[i].reshape(1,100)) for i in range(QA.shape[0])]).reshape(QA.shape[0],1)
    loc = np.argmax(ranks)  
    a = QA_dict[loc][1]
    print("A: ",a)

Q: can animals spread the virus


C:\Users\uroot\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


A:  At this time, CDC has no data to suggest that this new coronavirus or other similar coronaviruses are spread by
mosquitoes or ticks. The main way that COVID-19 spreads is from person to person. See How Coronavirus Spreads for
more information.

Q: can the virus cause depression


C:\Users\uroot\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


A:  At this time, CDC has no data to suggest that this new coronavirus or other similar coronaviruses are spread by
mosquitoes or ticks. The main way that COVID-19 spreads is from person to person. See How Coronavirus Spreads for
more information.

Q: when will it end


C:\Users\uroot\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


A:  Testing is particularly important for people who are seriously ill. Knowing the diagnosis is important for clinical care, allows health care workers to protect themselves, and is necessary for research into treatments.

For people who are mildly or moderately ill, testing can help assure that they isolate themselves and alert people they have come in contact with of the potential need for quarantine. By quickly identifying individuals who are sick and isolating them, public health authorities can reduce the spread of the novel coronavirus.

To assure both that tests are available for those who need them and that the health professionals are prepared to do the test safely, adequate safety equipment is essential. Not every clinic or medical office will conduct the testing. Physicians and public health authorities should direct people to where they can be tested. Given the limited availability of testing, there is much less urgency to test people who are feeling well.

Also, wherever 

C:\Users\uroot\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


A:  The virus that causes COVID-19 is thought to spread mainly from person to person, mainly through respiratory
droplets produced when an infected person coughs or sneezes. These droplets can land in the mouths or noses of
people who are nearby or possibly be inhaled into the lungs. Spread is more likely when people are in close contact
with one another (within about 6 feet).
COVID-19 seems to be spreading easily and sustainably in the community (“community spread”) in many a ected
geographic areas. Community spread means people have been infected with the virus in an area, including some who
are not sure how or where they became infected.

